This Project is in continuation of the previous project 1 and here we will be discussing about text generation at word using LSTM:
* Word level text generation
* Use of custom data like new articles

In [1]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run/code/Users/soutrik.chowdhury/advanced_pytorch_nlp"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run/code/Users/soutrik.chowdhury/advanced_pytorch_nlp


In [3]:
os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [4]:
import torch
from torchinfo import summary
import torchtext
import os
import json
from collections import Counter
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import ngrams_iterator
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import re
from torchview import draw_graph
from torcheval.metrics import MulticlassAccuracy, BinaryAccuracy
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix
import scikitplot as skplt
from src.pytorch_helpers import *
from torch.optim import Adam

plt.rcParams["figure.figsize"] = (25, 20)

In [5]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", 100)
pd.set_option("display.max_colwidth", 100)
import re
import string
from string import digits

In [6]:
# Set manual seed since nn.Parameter are randomly initialzied
set_seed(42)
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
epochs = 2
lr = 1e-3
batch_size = 64
input_len = 128

Random seed set as 42
cuda


In [7]:
json_data = json.load(open("./data/bloomberg_quint_news.json", "r"))
print(len(json_data))

7018


In [8]:
news_df = pd.DataFrame(json_data)

In [9]:
news_df.head(3)

,url,title,short_description,author,date_created,date_modified,category,raw_description,description,publisher,scraped_at
0,https://www.bloombergquint.com/markets/all-you-need-to-know-going-into-trade-on-september-23-2,All You Need To Know Going Into Trade On September 23,"Stocks in the news, big brokerage calls of the day, complete trade setup and much more!",Darshan A Nakhwa,"23 Sep 2021, 7:05 AM IST","23 Sep 2021, 7:05 AM IST",Markets,"<div class=""story-element story-element-text""><div>Asian stocks were steady early Thursday after...",Asian stocks were steady early Thursday after U.S. shares took in their stride the prospect of a...,https://www.bloombergquint.com/,2021-09-24 00:01:25
1,https://www.bloombergquint.com/business/bridgestone-ceo-backs-safe-tokyo-olympics-dials-back-mar...,"Bridgestone CEO Backs Safe Tokyo Olympics, Dials Back Marketing","Bridgestone CEO Backs Safe Tokyo Olympics, Dials Back Marketing",Shiho Takezawa &,"23 Apr 2021, 5:35 AM IST","23 Apr 2021, 6:35 AM IST",Business,"<div class=""story-element story-element-text""><div>(Bloomberg) -- Bridgestone Corp. will support...","Bridgestone Corp. will support the Tokyo Olympics and Paralympic games as a , as long as the s...",https://www.bloombergquint.com/,2021-09-24 00:01:26
2,https://www.bloombergquint.com/markets/stocks-to-watch-hcl-tech-cyient-mm-financial-services-tat...,"Stocks To Watch: HCL Tech, Cyient, M&M Financial Services, Tata Motors, Torrent Power, Wipro",Here are the stocks to watch in trade today...,BQ Desk,"23 Apr 2021, 7:29 AM IST","23 Apr 2021, 7:29 AM IST",Markets,"<div class=""story-element story-element-text""><div>Indian equity benchmarks reversed losses made...",Indian equity benchmarks reversed losses made in the first half of the session to snap a two-ses...,https://www.bloombergquint.com/,2021-09-24 00:01:26


In [10]:
def basic_cleanup(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = text.strip()
    return text

In [11]:
news_df["description"] = news_df["description"].apply(basic_cleanup)
news_df["title"] = news_df["title"].apply(basic_cleanup)
news_df["short_description"] = news_df["short_description"].apply(basic_cleanup)

In [12]:
# Consolidating text in one field only
news_df["Complete_Text"] = (
    news_df["title"] + " : " + news_df["short_description"] + " : " + news_df["description"]
)

In [13]:
# news_df["Complete_Text"].iloc[0]

In [14]:
news_dump = " ".join(news_df["Complete_Text"].tolist())

In [15]:
class TorchVocabulary:
    """
    Class to create the vocabulary for the text data

    Args:
    freq_threshold: int: frequency threshold for the words
    max_size: int: maximum size of the vocabulary
    tokenizer: tokenizer: tokenizer to tokenize the text

    """

    def __init__(self, freq_threshold, max_size, tokenizer) -> None:
        self.freq_threshold = freq_threshold
        self.max_size = max_size

        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {v: k for k, v in self.itos.items()}

        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.itos)

    def tokenization(self, text):
        return self.tokenizer(text)

    def build_vocabulary(self, sentence_list: list):
        """Build the vocabulary from the sentence list"""
        # calculate the frequencies of each word first to remove the words with freq < freq_threshold
        overall_word_counts = Counter()  # init the freq dict
        idx = 4  # index from which we want our dict to start. We already used 4 indexes for pad, start, end, unk

        for sentence in sentence_list:
            sentence_word_counts = Counter(self.tokenization(sentence))
            # Update the overall word counts
            overall_word_counts.update(sentence_word_counts)

        # remove low freq words
        frequencies = {
            k: v for k, v in overall_word_counts.items() if v >= self.freq_threshold
        }

        # limit the vocab size to max_size
        frequencies = dict(
            sorted(frequencies.items(), key=lambda x: -x[1])[: self.max_size]
        )

        # update stoi and itos post index 4 for the new words
        for word in frequencies:
            self.stoi[word] = idx
            self.itos[idx] = word
            idx += 1

    def create_text_vectors(self, text):
        """Convert the text to vectors"""
        tokens = self.tokenization(text)
        vec = [self.stoi.get(token, self.stoi["<UNK>"]) for token in tokens]
        return vec

In [16]:
vocabulary = TorchVocabulary(1, 100000, get_tokenizer("basic_english"))
vocabulary.build_vocabulary([news_dump])

In [17]:
len(vocabulary) # 4 more than the actual

57497

In [18]:
# breaking the data into samples of fixed size
token_ls = vocabulary.tokenization(news_dump)
data_sample = [
    token_ls[i : (i + input_len + 1)] for i in range(0, len(token_ls) - batch_size)
]

In [19]:
' '.join(data_sample[0])

'all you need to know going into trade on september 23 stocks in the news big brokerage calls of the day complete trade setup and much more asian stocks were steady early thursday after us shares took in their stride the prospect of a reduction in federal reserve stimulus as early as november the treasury yield curve flattened stocks rose in australia and dipped in south korea japan is shut for a holiday and hong kong will reopen from one us futures climbed after the sp 500 pushed higher for the first time in five sessions and the nasdaq 100 advanced fed chair jerome powell said the central bank could begin scaling back asset purchases in november and complete the process by mid2022 officials also revealed a growing inclination'

In [20]:
' '.join(data_sample[1])

'you need to know going into trade on september 23 stocks in the news big brokerage calls of the day complete trade setup and much more asian stocks were steady early thursday after us shares took in their stride the prospect of a reduction in federal reserve stimulus as early as november the treasury yield curve flattened stocks rose in australia and dipped in south korea japan is shut for a holiday and hong kong will reopen from one us futures climbed after the sp 500 pushed higher for the first time in five sessions and the nasdaq 100 advanced fed chair jerome powell said the central bank could begin scaling back asset purchases in november and complete the process by mid2022 officials also revealed a growing inclination to'

In [21]:
len(data_sample[0]), len(data_sample[1])

(129, 129)

In [22]:
len(set(token_ls))

57493

In [23]:
min([len(x) for x in data_sample])

65

In [24]:
class NewsDataset(Dataset):

    def __init__(self, samples, vocabulary):
        self.samples = samples
        self.vocabulary = vocabulary

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        samples = self.samples[idx]
        input_txt = torch.tensor(
            self.vocabulary.create_text_vectors(" ".join(samples[:-1])),
            dtype=torch.long,
        ).to(device)

        target_txt = torch.tensor(
            self.vocabulary.create_text_vectors(" ".join(samples[1:])), dtype=torch.long
        ).to(device)

        return input_txt, target_txt

In [25]:
train_dataset = NewsDataset(data_sample, vocabulary)

In [26]:
x, y = next(iter(train_dataset))
x.shape, y.shape

(torch.Size([128]), torch.Size([128]))

In [27]:
train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, drop_last=True, shuffle=False
)

In [28]:
for x, y in train_dataloader:
    print(x.shape, y.shape)
    break

torch.Size([64, 128]) torch.Size([64, 128])


In [29]:
y.view(-1).shape

torch.Size([8192])

In [30]:
class TextGenerationLSTM(nn.Module):

    def __init__(
        self, vocab_size, embedding_dim, hidden_dim, num_layers, drop_out=0.2
    ) -> None:
        super(TextGenerationLSTM, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.embedding_layer = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.lstm_layer = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True,
        )
        self.fc = nn.Linear(self.hidden_dim, self.vocab_size)

    def init_hidden(self, batch_size):
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        return h0, c0

    def forward(self, x):
        hidden_states = self.init_hidden(x.size(0))
        embedded = self.embedding_layer(x)
        out, (h_n, c_n) = self.lstm_layer(embedded, hidden_states)
        # print(out.shape)
        out = out.contiguous().view(-1, self.hidden_dim)
        # print(out.shape)
        out = self.fc(out)
        return out, (h_n, c_n)

In [31]:
word_lstm_model = TextGenerationLSTM(len(vocabulary), 100, 256, 2).to(device)
word_lstm_model

TextGenerationLSTM(
  (embedding_layer): Embedding(57497, 100)
  (lstm_layer): LSTM(100, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=57497, bias=True)
)

In [32]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

In [33]:
def train_module(
    model: torch.nn.Module,
    device: torch.device,
    train_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    train_losses: list,
):

    # setting model to train mode
    model.train()
    pbar = tqdm(train_dataloader)

    # batch metrics
    train_loss = 0
    processed_batch = 0

    for idx, (data, label) in enumerate(pbar):
        # setting up device
        data = data.to(device)
        label = label.to(device)

        # forward pass output
        preds, _ = model(data)

        # calc loss
        loss = criterion(preds, label.view(-1))
        train_loss += loss.item()
        # print(f"training loss for batch {idx} is {loss}")

        # backpropagation
        optimizer.zero_grad()  # flush out  existing grads
        loss.backward()  # back prop of weights wrt loss
        optimizer.step()  # optimizer step -> minima

        # updating batch count
        processed_batch += 1

        pbar.set_description(f"Avg Train Loss: {train_loss/processed_batch}")

    # updating epoch metrics
    train_losses.append(train_loss / processed_batch)

    return train_losses

In [34]:
def test_module(
    model: torch.nn.Module,
    device: torch.device,
    test_dataloader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
    test_losses,
):
    # setting model to eval mode
    model.eval()
    pbar = tqdm(test_dataloader)

    # batch metrics
    test_loss = 0
    processed_batch = 0

    with torch.inference_mode():
        for idx, (data, label) in enumerate(pbar):
            data, label = data.to(device), label.to(device)
            # predictions
            preds = model(data)
            # print(preds.shape)
            # print(label.shape)

            # loss calc
            loss = criterion(preds, label)
            test_loss += loss.item()

            # updating batch count
            processed_batch += 1

            pbar.set_description(f"Avg Test Loss: {test_loss/processed_batch} ")

        # updating epoch metrics
        test_losses.append(test_loss / processed_batch)

    return test_losses

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(word_lstm_model.parameters(), lr=lr)

In [36]:
# Place holders----
train_losses = []
test_losses = []

for epoch in range(0, epochs):
    print(f"Epoch {epoch}")
    train_losses, train_metrics = train_module(
        word_lstm_model,
        device,
        train_dataloader,
        optimizer,
        criterion,
        train_losses,
    )

Epoch 0


  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | 0/32481 [00:00<?, ?it/s]

Avg Train Loss: 6.7252771457794855: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 32480/32481 [1:09:21<00:00,  7.80it/s]


RuntimeError: stack expects each tensor to be equal size, but got [128] at entry 0 and [127] at entry 55